In [ ]:
from hail import *
hc = HailContext()

In [ ]:
vcf = 'hail-tutorial-files/1000Genomes.ALL.coreExome10K-v1.vcf.bgz'
sample_annotations = 'hail-tutorial-files/1000Genomes.ALL.coreExome10K-v1.sample_annotations'
pruned_variants = 'hail-tutorial-files/purcell5k.interval_list'

In [ ]:
vds = hc.import_vcf(vcf)
vds = vds.split_multi()
vds = vds.annotate_samples_table(sample_annotations, root='sa.pheno',sample_expr='Sample', config=TextTableConfig(impute=True))
out_path = 'out/1kg.vds'
vds.write(out_path)

In [ ]:
vds = hc.read(out_path)
vds.count(genotypes=True)

In [ ]:
filter_condition = '''let ab = g.ad[1] / g.ad.sum in
                          ((g.isHomRef && ab <= 0.1) || 
                           (g.isHet && ab >= 0.25 && ab <= 0.75) || 
                           (g.isHomVar && ab >= 0.9))'''
filtered_vds = vds.filter_genotypes(filter_condition)
filtered_vds.count(genotypes=True)

In [ ]:
filtered_vds_2 = (filtered_vds
     .filter_variants_expr('gs.fraction(g => g.isCalled) > 0.95')
     .sample_qc())
filtered_vds_2.print_schema(sa=True)
filtered_vds_2.export_samples('out/sampleqc.txt', 'Sample = s.id, sa.qc.*')

In [ ]:
%%sh
head sampleqc.txt | cut -f 1,2,3,4,5,6,7,8,9,10

In [ ]:
(filtered_vds_2
      .filter_samples_expr('sa.qc.callRate >= 0.97 && sa.qc.gqMean >= 20')
      .export_samples('out/included_samples.txt', 's.id'))
filtered_vds_3 = filtered_vds.filter_samples_list('out/included_samples.txt')
print 'before filter: %d samples' % filtered_vds.num_samples()
print 'after filter: %d samples' % filtered_vds_3.num_samples()
method_1_kept_ids = filtered_vds_3.sample_ids()

filtered_vds_3 = (filtered_vds
    .annotate_samples_table('out/sampleqc.txt', sample_expr='Sample', 
                            root='sa.qc', config=TextTableConfig(impute=True))
    .filter_samples_expr('sa.qc.callRate >= 0.97 && sa.qc.gqMean >= 20'))
print 'before filter: %d samples' % filtered_vds.num_samples()
print 'after filter: %d samples' % filtered_vds_3.num_samples()
method_2_kept_ids = filtered_vds_3.sample_ids()

method_1_kept_ids == method_2_kept_ids

post_qc_exprs = [
    'global.postQC.nCases = samples.filter(s => sa.pheno.PurpleHair).count()',
    'global.postQC.nControls = samples.filter(s => !sa.pheno.PurpleHair).count()' ]
filtered_vds_3.annotate_global_expr_by_sample(post_qc_exprs).show_globals()

filtered_vds_3 = filtered_vds_3.variant_qc()
filtered_vds_3.print_schema(va=True)
filtered_vds_3.export_variants('out/variantqc.tsv',
                               'Chrom=v.contig, Pos=v.start, Ref=v.ref, Alt=v.alt, va.qc.*')

In [ ]:
hwe_expressions = [
    'va.hweByPop.hweEUR = gs.filter(g => sa.pheno.SuperPopulation == "EUR").hardyWeinberg()',
    'va.hweByPop.hweSAS = gs.filter(g => sa.pheno.SuperPopulation == "SAS").hardyWeinberg()',
    'va.hweByPop.hweAMR = gs.filter(g => sa.pheno.SuperPopulation == "AMR").hardyWeinberg()',
    'va.hweByPop.hweAFR = gs.filter(g => sa.pheno.SuperPopulation == "AFR").hardyWeinberg()',
    'va.hweByPop.hweEAS = gs.filter(g => sa.pheno.SuperPopulation == "EAS").hardyWeinberg()' ]
filtered_vds_3 = filtered_vds_3.annotate_variants_expr(hwe_expressions)
filtered_vds_3.persist()
filtered_vds_3.print_schema(va=True)

hwe_filter_expression = '''
    va.hweByPop.hweEUR.pHWE > 1e-6 && 
    va.hweByPop.hweSAS.pHWE > 1e-6 && 
    va.hweByPop.hweAMR.pHWE > 1e-6 && 
    va.hweByPop.hweAFR.pHWE > 1e-6 && 
    va.hweByPop.hweEAS.pHWE > 1e-6 '''
hwe_filtered_vds = filtered_vds_3.filter_variants_expr(hwe_filter_expression)
hwe_filtered_vds.count()

In [ ]:
vds.filter_variants_expr('v.contig == "X"').num_variants()

In [ ]:
final_filtered_vds = hwe_filtered_vds.filter_variants_expr('va.qc.gqMean >= 20')
final_filtered_vds.count()

In [ ]:
sex_aware_hwe_exprs = [ 
     '''va.hweByPop.hweEUR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "EUR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "EUR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweSAS = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "SAS").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "SAS" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweAMR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "AMR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "AMR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweAFR = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "AFR").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "AFR" && sa.pheno.isFemale).hardyWeinberg()''',
     '''va.hweByPop.hweEAS = 
        if (v.contig != "X") 
          gs.filter(g => sa.pheno.SuperPopulation == "EAS").hardyWeinberg() 
        else 
          gs.filter(g => sa.pheno.SuperPopulation == "EAS" && sa.pheno.isFemale).hardyWeinberg()''' ]
hwe_filtered_vds_fixed = (filtered_vds_3
    .annotate_variants_expr(sex_aware_hwe_exprs)
    .filter_variants_expr(hwe_filter_expression)
    .persist())
print 'total variants = %s' % hwe_filtered_vds_fixed.num_variants()
print 'X chromosome variants = %s' % hwe_filtered_vds_fixed.filter_variants_expr('v.contig == "X"').num_variants()

In [ ]:
sex_check_vds = (hwe_filtered_vds_fixed
    .impute_sex(maf_threshold=0.05)
    .annotate_samples_expr('sa.sexcheck = sa.pheno.isFemale == sa.imputesex.isFemale'))
total_samples = sex_check_vds.num_samples()
sex_check_passes = sex_check_vds.filter_samples_expr('sa.sexcheck').num_samples()
print 'total samples: %s' % total_samples
print 'sex_check_passes: %s' % sex_check_passes

(sex_check_vds.annotate_global_expr_by_sample(
    'global.sexcheckCounter = samples.map(s => sa.sexcheck).counter()')
    .show_globals())

sex_check_filtered_vds = sex_check_vds.filter_samples_expr('sa.sexcheck || isMissing(sa.sexcheck)').persist()
print 'samples after filter: %s' % sex_check_filtered_vds.num_samples()

In [ ]:
pca_vds = (sex_check_filtered_vds.filter_variants_intervals('hail-tutorial-files/purcell5k.interval_list')
    .pca(scores='sa.pca'))
pca_vds.export_samples('out/pcaPlusPopulation.tsv', 
    'Sample=s, SuperPopulation=sa.pheno.SuperPopulation,'
    'Population=sa.pheno.Population, sa.pca.*')